In [20]:
# 필요한 라이브러리 설치
!pip install datasets torch torchvision scikit-learn matplotlib
!git clone https://github.com/HRNet/HRNet-Image-Classification
%cd HRNet-Image-Classification
!pip install -r requirements.txt
%cd ..

!wget -O /content/HRNet_W48_C_ssld_pretrained.pth https://github.com/HRNet/HRNet-Image-Classification/releases/download/PretrainedWeights/HRNet_W48_C_ssld_pretrained.pth
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from datasets import load_dataset
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns

fatal: destination path 'HRNet-Image-Classification' already exists and is not an empty directory.
/content/HRNet-Image-Classification
  Using cached easydict-1.7.tar.gz (6.2 kB)
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==3.4.1.15 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86)
ERROR: No matching distribution found for opencv-python==3.4.1.15
/content
--2025-05-03 09:36:42--  https://github.com/HRNet/HRNet-Image-Classification/releases/download/PretrainedWeights/HRNet_W48_C_ssld_pretrained.pth
Resolving github.com (github.com)... 20.205.243.166
Conne

In [21]:
# HRNet 모델 임포트
import sys
sys.path.append('HRNet-Image-Classification/lib')
from models.cls_hrnet import get_cls_net

# 데이터셋 로드
dataset = load_dataset("dwb2023/brain-tumor-image-dataset-semantic-segmentation")
train = dataset["train"]
test = dataset["test"]
valid = dataset["valid"]

# 이미지 전처리 파이프라인 (강화된 증강)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [22]:
# 커스텀 데이터셋 클래스
class BrainTumorDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        label = self.dataset[idx]["category_id"]  # 원본 유지 (1=Tumor, 2=Normal)
        if self.transform:
            image = self.transform(image)
        # 레이블을 [0,1] 범위로 조정하지 않고 원본 값 유지
        return image, torch.tensor(label, dtype=torch.float32)

# 데이터로더 생성
train_dataset = BrainTumorDataset(train, transform=transform)
valid_dataset = BrainTumorDataset(valid, transform=transform)
test_dataset = BrainTumorDataset(test, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [23]:
# HRNet-W48-C-ssld 공식 모델 구성
hrnet_config = {
    'MODEL': {
        'NAME': 'cls_hrnet_w48',
        'EXTRA': {
            'STAGE1': {
                'NUM_MODULES': 1,
                'NUM_BRANCHES': 1,
                'BLOCK': 'BOTTLENECK',
                'NUM_BLOCKS': [4],
                'NUM_CHANNELS': [64],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE2': {
                'NUM_MODULES': 1,
                'NUM_BRANCHES': 2,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4],
                'NUM_CHANNELS': [48, 96],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE3': {
                'NUM_MODULES': 4,
                'NUM_BRANCHES': 3,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4,4],
                'NUM_CHANNELS': [48, 96, 192],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE4': {
                'NUM_MODULES': 3,
                'NUM_BRANCHES': 4,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4,4,4],
                'NUM_CHANNELS': [48, 96, 192, 384],
                'FUSE_METHOD': 'SUM'
            }
        },
        'NUM_CLASSES': 1000
    }
}

# 모델 초기화 및 사전학습 가중치 로드
model = get_cls_net(hrnet_config)
pretrained_dict = torch.load('/content/HRNet_W48_C_ssld_pretrained.pth')
model.load_state_dict(pretrained_dict, strict=False)

# 안정적인 분류기 구조 (2-layer MLP)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1)
)

In [24]:
# Focal Loss 정의 (클래스 불균형 대응)
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.8, gamma=2.0, eps=1e-6):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        pt = torch.clamp(pt, self.eps, 1.0 - self.eps)  # NaN 방지
        loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return loss.mean()

# 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = FocalLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Gradient Clipping 함수
def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)


In [26]:
# 학습 루프
num_epochs = 10
best_val_auc = 0
grad_clip = 1.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    total_samples = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()

        # 레이블 변환: 2(Normal)→0, 1(Tumor)→1
        labels_binary = torch.where(labels == 1, 1.0, 0.0)
        loss = criterion(outputs, labels_binary)

        loss.backward()
        clip_gradient(optimizer, grad_clip)
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

    train_loss /= total_samples
    scheduler.step()

    # 검증 단계
    model.eval()
    val_probs, val_labels = [], []
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            val_probs.extend(torch.sigmoid(outputs).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # 레이블 변환: 2→0, 1→1
    val_labels_binary = np.where(np.array(val_labels) == 1, 1, 0)
    val_preds = (np.array(val_probs) > 0.5).astype(int)

    # 평가 지표 계산
    val_f1 = f1_score(val_labels_binary, val_preds)
    val_recall = recall_score(val_labels_binary, val_preds)
    val_precision = precision_score(val_labels_binary, val_preds)
    val_auc = roc_auc_score(val_labels_binary, val_probs)
    tn, fp, fn, tp = confusion_matrix(val_labels_binary, val_preds).ravel()
    val_iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f} | Val Recall: {val_recall:.4f}")
    print(f"Val Precision: {val_precision:.4f} | Val IoU: {val_iou:.4f} | Val AUC: {val_auc:.4f}")

    # 최고 모델 저장
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss,
            'auc': val_auc
        }, 'best_model.pth')
        print(f"New best model saved with AUC: {val_auc:.4f}")

# 테스트 평가
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

test_probs, test_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        test_probs.extend(torch.sigmoid(outputs).cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# 레이블 변환: 2→0, 1→1
test_labels_binary = np.where(np.array(test_labels) == 1, 1, 0)
test_preds = (np.array(test_probs) > 0.5).astype(int)

# 평가 지표 계산
test_f1 = f1_score(test_labels_binary, test_preds)
test_recall = recall_score(test_labels_binary, test_preds)
test_precision = precision_score(test_labels_binary, test_preds)
test_auc = roc_auc_score(test_labels_binary, test_probs)
tn, fp, fn, tp = confusion_matrix(test_labels_binary, test_preds).ravel()
test_iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0

# 최종 결과 출력
print("\n=== 최종 테스트 성능 ===")
print(f"F1-score: {test_f1:.4f}")
print(f"Recall (Tumor): {test_recall:.4f} [TP:{tp}/FN:{fn}]")
print(f"Precision (Tumor): {test_precision:.4f} [TP:{tp}/FP:{fp}]")
print(f"IoU (Tumor): {test_iou:.4f}")
print(f"ROC-AUC: {test_auc:.4f}")

# 혼동 행렬 시각화
conf_mat = confusion_matrix(test_labels_binary, test_preds)
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal','Tumor'],
            yticklabels=['Normal','Tumor'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix\nRecall: {test_recall:.2%}, Precision: {test_precision:.2%}')
plt.show()


Epoch 1/10
Train Loss: 0.0080 | Val F1: 0.9766 | Val Recall: 0.9952
Val Precision: 0.9587 | Val IoU: 0.9543 | Val AUC: 0.9973
New best model saved with AUC: 0.9973

Epoch 2/10
Train Loss: 0.0051 | Val F1: 0.9789 | Val Recall: 0.9952
Val Precision: 0.9631 | Val IoU: 0.9587 | Val AUC: 0.9992
New best model saved with AUC: 0.9992

Epoch 3/10
Train Loss: 0.0022 | Val F1: 0.9904 | Val Recall: 0.9810
Val Precision: 1.0000 | Val IoU: 0.9810 | Val AUC: 0.9975

Epoch 4/10
Train Loss: 0.0015 | Val F1: 0.9952 | Val Recall: 0.9905
Val Precision: 1.0000 | Val IoU: 0.9905 | Val AUC: 0.9988

Epoch 5/10
Train Loss: 0.0012 | Val F1: 0.9952 | Val Recall: 0.9905
Val Precision: 1.0000 | Val IoU: 0.9905 | Val AUC: 0.9997
New best model saved with AUC: 0.9997

Epoch 6/10
Train Loss: 0.0019 | Val F1: 0.9976 | Val Recall: 0.9952
Val Precision: 1.0000 | Val IoU: 0.9952 | Val AUC: 0.9993

Epoch 7/10
Train Loss: 0.0009 | Val F1: 0.9928 | Val Recall: 0.9905
Val Precision: 0.9952 | Val IoU: 0.9858 | Val AUC: 0.99

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy._core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weig